In [1]:
import os
import array
import numpy as np

In [2]:
# Hengen lab neuraltoolkit code to read an e-cube file

# f = open(name, 'rb')
# tr = np.fromfile(f, dtype=np.uint64, count=1)
# dr = np.fromfile(f, dtype=np.int16,  count=-1)
# length = np.int64(np.size(dr)/number_of_channels)
# drr = np.reshape(dr, [number_of_channels, length], order='F')
# f.close()
# return tr, drr

In [3]:
# Read the first few integers of the file as raw ints with no abstractions in the way
data = array.array("h")  # signed short

with open("/public/groups/braingeneers/staging/original/Carbon_EAB27_Headstages_64_Channels_int16_2018-12-28_14-55-21.bin", "rb") as fd:
    fd.seek(8, os.SEEK_SET)  # Skip timestamp
    data.fromfile(fd, 8)
    
data.tolist()

[-104, -1288, -1052, -432, -284, -800, 60, -4248]

In [4]:
# Read in raw binary data from Hengen lab and interpret as they do in the neuraltoolkit
with open("/public/groups/braingeneers/staging/original/Carbon_EAB27_Headstages_64_Channels_int16_2018-12-28_14-55-21.bin") as fd:
    fd.seek(8, os.SEEK_SET)
    raw = np.fromfile(fd, dtype=np.int16)
    
print(raw[0:8])
print(raw.flags)

[ -104 -1288 -1052  -432  -284  -800    60 -4248]
  C_CONTIGUOUS : True
  F_CONTIGUOUS : True
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False


In [5]:
# Reshape as per the neuraltoolkit code that Hengen lab uses. - Note F_CONTIGUOUS
column_major = raw.reshape([64, -1], order="F")
print(column_major.shape)
print(column_major.flags)
print(column_major[:, 0])

(64, 7500584)
  C_CONTIGUOUS : False
  F_CONTIGUOUS : True
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False
[ -104 -1288 -1052  -432  -284  -800    60 -4248   100 -1864  -692  -480
   136  -616  -672 -1056 -5312  -468  -584 -1024   840  -568  -532  -488
  -252  -856  -488  -856   -44  -268 -1056 -1008   -60  -536    56  -412
   -68 -1368  -712  -488  -192 -1128  -672  -856  -608 -1080  -472  -532
  -808    -8  -328  -192  -824  -420 -1352  -304  -508 -1348   304  -836
 -3612  -740  -708 -1112]


In [6]:
# Reshape using "C" interpretation - note C_CONTIGUOUS
row_major = raw.reshape([-1, 64], order="C")
print(row_major.shape)
print(row_major.flags)
print(row_major[0, :])

(7500584, 64)
  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False
[ -104 -1288 -1052  -432  -284  -800    60 -4248   100 -1864  -692  -480
   136  -616  -672 -1056 -5312  -468  -584 -1024   840  -568  -532  -488
  -252  -856  -488  -856   -44  -268 -1056 -1008   -60  -536    56  -412
   -68 -1368  -712  -488  -192 -1128  -672  -856  -608 -1080  -472  -532
  -808    -8  -328  -192  -824  -420 -1352  -304  -508 -1348   304  -836
 -3612  -740  -708 -1112]


In [7]:
# Install the braingeneers python package - force upgrade in case another notebook has somethign open
!pip install --user --quiet --upgrade git+https://github.com/braingeneers/braingeneerspy.git@e37f639
import braingeneers.datasets

You should consider upgrading via the 'pip install --upgrade pip' command.


In [9]:
batch_uuid = "2019-08-Hengen-Mouse"
batch = braingeneers.datasets.load_batch(batch_uuid)
print(batch["notes"])

experiment = braingeneers.datasets.load_experiment(batch_uuid, 0)
print(experiment["notes"])

Mouse recordings from the Hengen lab using various probe types
Mouse recording from Hengen lab using carbon probe


In [10]:
block, t, fs = braingeneers.datasets.load_blocks(batch_uuid, experiment_num=0, start=0, stop=1)

In [11]:
block[0, :] / experiment["scaler"]

array([ -104., -1288., -1052.,  -432.,  -284.,  -800.,    60., -4248.,
         100., -1864.,  -692.,  -480.,   136.,  -616.,  -672., -1056.,
       -5312.,  -468.,  -584., -1024.,   840.,  -568.,  -532.,  -488.,
        -252.,  -856.,  -488.,  -856.,   -44.,  -268., -1056., -1008.,
         -60.,  -536.,    56.,  -412.,   -68., -1368.,  -712.,  -488.,
        -192., -1128.,  -672.,  -856.,  -608., -1080.,  -472.,  -532.,
        -808.,    -8.,  -328.,  -192.,  -824.,  -420., -1352.,  -304.,
        -508., -1348.,   304.,  -836., -3612.,  -740.,  -708., -1112.],
      dtype=float32)